In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
import os
os.chdir("../")
%pwd


'/home/shiv/projects/Chest-Cancer-CNN-Classifier-with-MLFLOW-and-DVC'

In [2]:
import dagshub
dagshub.init(repo_owner='bhavin2004', repo_name='Chest-Cancer-CNN-Classifier-with-MLFLOW-and-DVC', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)
  

Accessing as bhavin2004

Initialized MLflow to track repo "bhavin2004/Chest-Cancer-CNN-Classifier-with-MLFLOW-and-DVC"

Repository bhavin2004/Chest-Cancer-CNN-Classifier-with-MLFLOW-and-DVC initialized!

🏃 View run charming-goose-352 at: https://dagshub.com/bhavin2004/Chest-Cancer-CNN-Classifier-with-MLFLOW-and-DVC.mlflow/#/experiments/0/runs/6e6c1f8f0bf546b39e777284b35d4efe
🧪 View experiment at: https://dagshub.com/bhavin2004/Chest-Cancer-CNN-Classifier-with-MLFLOW-and-DVC.mlflow/#/experiments/0


In [4]:

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)
  print('hi')
  

hi
🏃 View run mercurial-doe-718 at: https://dagshub.com/bhavin2004/Chest-Cancer-CNN-Classifier-with-MLFLOW-and-DVC.mlflow/#/experiments/0/runs/8507b8927a184d8b8c05bed5b19883bf
🧪 View experiment at: https://dagshub.com/bhavin2004/Chest-Cancer-CNN-Classifier-with-MLFLOW-and-DVC.mlflow/#/experiments/0


In [5]:
import tensorflow as tf


2025-06-26 04:07:37.180600: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-26 04:07:37.239345: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750910857.256180    6572 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750910857.262091    6572 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-26 04:07:37.327872: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [8]:
model = tf.keras.models.load_model('artifacts/training/model.keras')

I0000 00:00:1750911014.618875    6572 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9
W0000 00:00:1750911014.996327    8098 gpu_backend_lib.cc:579] Can't find libdevice directory ${CUDA_DIR}/nvvm/libdevice. This may result in compilation or runtime failures, if the program we try to run uses routines from libdevice.
Searched for CUDA in the following directories:
  ./cuda_sdk_lib
  ipykernel_launcher.runfiles/cuda_nvcc
  ipykern/cuda_nvcc
  
  /usr/local/cuda
  /home/shiv/projects/Chest-Cancer-CNN-Classifier-with-MLFLOW-and-DVC/venv/lib/python3.12/site-packages/tensorflow/python/platform/../../../nvidia/cuda_nvcc
  /home/shiv/projects/Chest-Cancer-CNN-Classifier-with-MLFLOW-and-DVC/venv/lib/python3.12/site-packages/tensorflow/python/platform/../../../../nvidia/cuda_nvcc
  /home/shiv/projects/Chest-Cancer-CNN-Classifier-with-MLFL

In [15]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class EvaluationConfig:
    path_to_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [20]:
import os
from src.cnnclassifier.constants import *
from src.cnnclassifier.utils.common import read_yaml, create_directories,save_json

In [25]:
class ConfigurationManager:
    def __init__(
        self,
        confif_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH) -> None:
        
        self.config = read_yaml(confif_filepath)
        self.params= read_yaml(params_filepath)
        # print(type([self.config.artifacts_root]))
        create_directories([self.config.artifacts_root])
        
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_to_model=self.config.training.trained_model_path,
            training_data=Path(os.path.join(self.config.data_ingestion.unzip_dir,'Data')),
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE,
        )
        
        return eval_config

In [22]:
import tensorflow as tf
import mlflow
import dagshub
import mlflow.keras
import dagshub
from urllib.parse import urlparse


In [ ]:
import mlflow.keras



class Evaluation:
    def __init__(self,config=EvaluationConfig) :
        self.config=config
    
    def _valid_generator(self):
        
        datagenerator_kwargs = dict(
            rescale = 1./255,
            # validation_split = 0.2
        )
        
        dataflow_kwargs=  dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = 'bilinear'
        )
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=os.path.join(self.config.training_data,'valid'),
            shuffle = False,
            **dataflow_kwargs
        )
        
    @staticmethod
    def load_model(path: Path)-> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    
    def evaluaion(self):
        self.model = self.load_model(self.config.path_to_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        # self.score = {'loss': score[0],'accuracy':score[1]}
        self.save_score()
        
    def save_score(self):
        print("Saving score now...")
        print("Score:", self.score)
        score = {'loss': self.score[0],'accuracy':self.score[1]}
        save_json(path=Path('scores.json'), data=score)

        
   
    def log_into_mlflow(self):
        import dagshub
        dagshub.init(repo_owner='bhavin2004', repo_name='Chest-Cancer-CNN-Classifier-with-MLFLOW-and-DVC', mlflow=True)
        
        mlflow.set_tracking_uri("https://dagshub.com/bhavin2004/Chest-Cancer-CNN-Classifier-with-MLFLOW-and-DVC.mlflow")

        with mlflow.start_run():
            mlflow.log_params({
                "image_size": self.config.params_image_size,
                "batch_size": self.config.params_batch_size
            })

            mlflow.log_metrics({'loss': self.score[0],'accuracy':self.score[1]})
            mlflow.keras.log_model(self.model, "model")




In [70]:
from src.cnnclassifier import logger,CustomException
import sys

try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluaion()
    evaluation.log_into_mlflow()
    pass
except Exception as e:
    CustomException(e,sys)

[2025-06-26 05:47:08,005] [INFO] [common:77] - yaml file:config/config.yaml loaded successfully
[2025-06-26 05:47:08,007] [INFO] [common:77] - yaml file:params.yaml loaded successfully
[2025-06-26 05:47:08,008] [INFO] [common:95] - artifacts directory is created
Found 72 images belonging to 4 classes.


/home/shiv/projects/Chest-Cancer-CNN-Classifier-with-MLFLOW-and-DVC/venv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/home/shiv/projects/Chest-Cancer-CNN-Classifier-with-MLFLOW-and-DVC/venv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 114ms/step - accuracy: 0.5353 - loss: 1.2557
Saving score now...
Score: [1.233647108078003, 0.4444444477558136]
[2025-06-26 05:47:09,711] [ERROR] [__init__:62] - Error occurred in script: /tmp/ipykernel_6572/2584140842.py at line number: 8
And Error message: maximum recursion depth exceeded
